<a href="https://colab.research.google.com/github/meghansterenberg/CS167Code/blob/main/Copy_of_Notebook2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook #2: K Nearest Neighbors and Normalization
**Name: Meghan Sterenberg**


## Load in the data:
You'll notice that there are a lot of columns here. We're going to work with a subset of these columns, not the entire dataset.

In [2]:
import pandas
import numpy
from google.colab import drive 
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
data = pandas.read_csv('/content/drive/MyDrive/Cs167Datasets/LifeExpectancyData.csv')
data.head()

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62.0,0.01,71.279624,65.0,1154.0,19.1,83.0,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64.0,0.01,73.523582,62.0,492.0,18.6,86.0,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66.0,0.01,73.219243,64.0,430.0,18.1,89.0,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69.0,0.01,78.184215,67.0,2787.0,17.6,93.0,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71.0,0.01,7.097109,68.0,3013.0,17.2,97.0,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


## Exercise #1: 
**Work with the right subset**: 

You're not going to work with the whole data set, just the 'Status', 'Life expectancy' and 3-5 additional features of your choice. To start, make this subset of the original data. Also, make sure your subset contains only countries with the 'Status' feature as **Developing**.

*hint: your initial subset should have between 2,000 and 2,500 rows.*

In [21]:
data = data.copy()
developing_countries = data[(data['Status'] == 'Developing')]
subset = developing_countries [ ['Life expectancy', 'Status', 'Population', 'under-five deaths','Diphtheria', 'percentage expenditure']]
subset.shape

(2426, 6)

## Exercise #2:
**Check for null values**: If you have a null target value ('Life expectancy'), you will need to throw that example out. 

Replace each of the null values within your predictors with the average of the column.

*hint: Useful functions here are `isna()`, `any()`, `fillna()`, `value_counts()` and `dropna()`. If you're not sure what these do, look them up in the [pandas documentation](https://pandas.pydata.org/docs/).*

After you have eliminated null values, use `.isna().any()` to verify that your fields no longer contain any null values (all of the values should be False). 

In [23]:
print("before: ", subset['Life expectancy'].isna().any())
life_exp_mean = subset['Life expectancy'].mean()
subset['Life expectancy'].fillna(life_exp_mean, inplace=True)
print("after: ", subset['Life expectancy'].isna().any())

before:  True
after:  False


/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


## Exercise #3
Write up a k-nearest-neighbors function like the one you made for the iris data set in class. It should be able to make life expectancy predictions for new countries. You should also be able to specify what you want to use as k.

You will need to incorporate the predictor features that you have chosen to be a part of your subset in your knn function.

Pay special attention to whether this problem is a classification or a regression, and how the answer to that question may change the output of your model. 

In [29]:
def knn(specimen, data, k):
  data_copy = data.copy()
  data_copy['distance_to_new'] = numpy.sqrt(
    (specimen['percentage expenditure']-data['percentage expenditure'])**2
    +(specimen['Population']-data['Population'])**2
    +(specimen['under-five deaths']-data['under-five deaths'])**2
    +(specimen['Diphtheria']-data['Diphtheria'])**2)
  sorted_data = data_copy.sort_values(['distance_to_new'])
  top_k = sorted_data.iloc[0:k]
  print(top_k['Life expectancy'].mode()[0])

## Exercise #4
Demonstrate that your function works by making up some new values for hypothetical countries and using your function to display the predicted life expectancy for that country.


In [39]:
country = {}
country['percentage expenditure'] = 70.36
country['Population'] = 1687953458
country['under-five deaths'] = 2
country['Diphtheria'] = 52

knn(country,subset,5)

64.0


## Exercise #5

Make a copy of the data and normalize the training data using Z-score.

*hint: `.copy()` will be useful here.* 

Predict the life expectancy with some input parameters using your k-nearest-neighbors function with both the normalized and non-normalized training data. Compare your results. Use a markup cell to describe and explain the differences in a few sentences.

In [40]:
percentexpend_std = subset['percentage expenditure'].std()
percentexpend_avg = subset['percentage expenditure'].mean()
population_std = subset['Population'].std()
population_avg = subset['Population'].mean()
under_five_std = subset['under-five deaths'].std()
under_five_avg = subset['under-five deaths'].mean()
diphtheria_std = subset['Diphtheria'].std()
diphtheria_avg = subset['Diphtheria'].mean()

norm_specimen = {}
norm_specimen['percentage expenditure'] = (country['percentage expenditure'] - percentexpend_avg)/percentexpend_std
norm_specimen['Population'] = (country['Population'] - population_avg)/population_std
norm_specimen['under-five deaths'] = (country['under-five deaths'] - under_five_avg)/under_five_std
norm_specimen['Diphtheria'] = (country['Diphtheria'] - diphtheria_avg)/diphtheria_std

norm_data = subset.copy() 
norm_data['percentage expenditure'] = (subset['percentage expenditure'] - percentexpend_avg)/percentexpend_std
norm_data['Population'] = (subset['Population'] - population_avg)/population_std
norm_data['under-five deaths'] = (subset['under-five deaths'] - under_five_avg)/under_five_std
norm_data['Diphtheria'] = (subset['Diphtheria'] - diphtheria_avg)/diphtheria_std

knn(norm_specimen,norm_data, 3)

64.8
